In [13]:
from __future__ import print_function
import six
import sys
import os
from optparse import OptionParser
from keras.models import load_model, Model, model_from_json
from argparse import ArgumentParser
from keras import backend as K
from keras.layers import Input
import numpy as np
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)
import h5py
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import pandas as pd
from keras.optimizers import Adam, Nadam
from callbacks import all_callbacks
import pandas as pd
from keras.layers import Input
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import yaml
from train import parse_config, get_features

In [15]:
from collections import namedtuple
Option = namedtuple("MyStruct", "inputFile tree config jsonModel outputDir")

options = Option(
    inputFile = '../data/processed-pythia82-lhc13-all-pt1-50k-r1_h022_e0175_t220_nonu_truth.z',
    tree = 't_allpar_new',
    config = 'train_config_threelayer.yml',
    jsonModel = '../weights/KERAS_3layer.json',
    outputDir = '../3layerNN_weights/'
)

print("Loading configuration from", options.config)
config = open(options.config, 'r')
yamlConfig =  yaml.load(config, Loader=yaml.FullLoader)
print(yamlConfig)

Loading configuration from train_config_threelayer.yml
{'Inputs': ['j_zlogz', 'j_c1_b0_mmdt', 'j_c1_b1_mmdt', 'j_c1_b2_mmdt', 'j_c2_b1_mmdt', 'j_c2_b2_mmdt', 'j_d2_b1_mmdt', 'j_d2_b2_mmdt', 'j_d2_a1_b1_mmdt', 'j_d2_a1_b2_mmdt', 'j_m2_b1_mmdt', 'j_m2_b2_mmdt', 'j_n2_b1_mmdt', 'j_n2_b2_mmdt', 'j_mass_mmdt', 'j_multiplicity'], 'Labels': ['j_g', 'j_q', 'j_w', 'j_z', 'j_t'], 'KerasModel': 'three_layer_model', 'KerasModelRetrain': 'three_layer_model_constraint', 'KerasLoss': 'categorical_crossentropy', 'L1Reg': 0.0001, 'NormalizeInputs': 1, 'InputType': 'Dense'}


In [16]:
X_train_val, X_test, y_train_val, y_test, labels  = get_features(options, yamlConfig)
    
from models.models import three_layer_model
keras_model = three_layer_model(Input(shape=X_train_val.shape[1:]), y_train_val.shape[1], l1Reg=yamlConfig['L1Reg'] )

startlearningrate=0.0001
adam = Adam(lr=startlearningrate)
keras_model.compile(optimizer=adam, loss=[yamlConfig['KerasLoss']], metrics=['accuracy'])


callbacks=all_callbacks(stop_patience=1000, 
                        lr_factor=0.5,
                        lr_patience=10,
                        lr_epsilon=0.000001, 
                        lr_cooldown=2, 
                        lr_minimum=0.0000001,
                        outputDir=options.outputDir)

keras_model.fit(X_train_val, y_train_val, batch_size = 1024, epochs = 100,
                validation_split = 0.25, shuffle = True, callbacks = callbacks.callbacks)

Keys: <KeysViewHDF5 ['t_allpar_new']>
(986806,)
('j_ptfrac', 'j_pt', 'j_eta', 'j_mass', 'j_tau1_b1', 'j_tau2_b1', 'j_tau3_b1', 'j_tau1_b2', 'j_tau2_b2', 'j_tau3_b2', 'j_tau32_b1', 'j_tau32_b2', 'j_zlogz', 'j_c1_b0', 'j_c1_b1', 'j_c1_b2', 'j_c2_b1', 'j_c2_b2', 'j_d2_b1', 'j_d2_b2', 'j_d2_a1_b1', 'j_d2_a1_b2', 'j_m2_b1', 'j_m2_b2', 'j_n2_b1', 'j_n2_b2', 'j_tau1_b1_mmdt', 'j_tau2_b1_mmdt', 'j_tau3_b1_mmdt', 'j_tau1_b2_mmdt', 'j_tau2_b2_mmdt', 'j_tau3_b2_mmdt', 'j_tau32_b1_mmdt', 'j_tau32_b2_mmdt', 'j_c1_b0_mmdt', 'j_c1_b1_mmdt', 'j_c1_b2_mmdt', 'j_c2_b1_mmdt', 'j_c2_b2_mmdt', 'j_d2_b1_mmdt', 'j_d2_b2_mmdt', 'j_d2_a1_b1_mmdt', 'j_d2_a1_b2_mmdt', 'j_m2_b1_mmdt', 'j_m2_b2_mmdt', 'j_n2_b1_mmdt', 'j_n2_b2_mmdt', 'j_mass_trim', 'j_mass_mmdt', 'j_mass_prun', 'j_mass_sdb2', 'j_mass_sdm1', 'j_multiplicity', 'j_g', 'j_q', 'j_w', 'j_z', 'j_t', 'j_undef')


C:\Users\Owner\JupyterNotebooks\keras-training\train/..\models\models.py:73: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, activation="relu", kernel_initializer="lecun_uniform", name="fc1_relu", kernel_regularizer=<keras.reg...)`
  name='fc1_relu', W_regularizer=l1(l1Reg))(Inputs)
C:\Users\Owner\JupyterNotebooks\keras-training\train/..\models\models.py:75: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, activation="relu", kernel_initializer="lecun_uniform", name="fc2_relu", kernel_regularizer=<keras.reg...)`
  name='fc2_relu', W_regularizer=l1(l1Reg))(x)
C:\Users\Owner\JupyterNotebooks\keras-training\train/..\models\models.py:77: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, activation="relu", kernel_initializer="lecun_uniform", name="fc3_relu", kernel_regularizer=<keras.reg...)`
  name='fc3_relu', W_regularizer=l1(l1Reg))(x)
C:\Users\Owner\JupyterNotebooks\keras-training\train/..\models\models.py:79: UserWarning: Updat

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 592083 samples, validate on 197361 samples







Epoch 1/100
592083/592083 [==============================] - 7s 12us/step - loss: 1.3459 - acc: 0.4738 - val_loss: 1.1139 - val_acc: 0.6048

***callbacks***
saving losses to ../trained_weights/losses.log

Epoch 00001: val_loss improved from inf to 1.11388, saving model to ../trained_weights//KERAS_check_best_model.h5

Epoch 00001: val_loss improved from inf to 1.11388, saving model to ../trained_weights//KERAS_check_best_model_weights.h5

Epoch 00001: saving model to ../trained_weights//KERAS_check_model_last.h5

Epoch 00001: saving model to ../trained_weights//KERAS_check_model_last_weights.h5

***callbacks end***


Epoch 2/100
592083/592083 [==============================] - 6s 10us/step - loss: 1.0459 - acc: 0.6608 - val_loss: 0.9926 - val_acc: 0.6803

***callbacks***
saving losses to ../trained_weights/losses.log

Epoch 00002: v

592083/592083 [==============================] - 4s 8us/step - loss: 0.8299 - acc: 0.7282 - val_loss: 0.8255 - val_acc: 0.7299

***callbacks***
saving losses to ../trained_weights/losses.log

Epoch 00010: val_loss improved from 0.83109 to 0.82548, saving model to ../trained_weights//KERAS_check_best_model.h5

Epoch 00010: val_loss improved from 0.83109 to 0.82548, saving model to ../trained_weights//KERAS_check_best_model_weights.h5

Epoch 00010: saving model to ../trained_weights//KERAS_check_model_last.h5

Epoch 00010: saving model to ../trained_weights//KERAS_check_model_last_weights.h5

Epoch 00010: saving model to ../trained_weights//KERAS_check_model_epoch10.h5

***callbacks end***

Epoch 11/100
592083/592083 [==============================] - 5s 8us/step - loss: 0.8254 - acc: 0.7296 - val_loss: 0.8210 - val_acc: 0.7305

***callbacks***
saving losses to ../trained_weights/losses.log

Epoch 00011: val_loss improved from 0.82548 to 0.82095, saving model to ../trained_weights//KERAS

592083/592083 [==============================] - 6s 9us/step - loss: 0.7941 - acc: 0.7371 - val_loss: 0.7911 - val_acc: 0.7382

***callbacks***
saving losses to ../trained_weights/losses.log

Epoch 00023: val_loss improved from 0.79286 to 0.79108, saving model to ../trained_weights//KERAS_check_best_model.h5

Epoch 00023: val_loss improved from 0.79286 to 0.79108, saving model to ../trained_weights//KERAS_check_best_model_weights.h5

Epoch 00023: saving model to ../trained_weights//KERAS_check_model_last.h5

Epoch 00023: saving model to ../trained_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 24/100
592083/592083 [==============================] - 6s 10us/step - loss: 0.7923 - acc: 0.7376 - val_loss: 0.7897 - val_acc: 0.7388

***callbacks***
saving losses to ../trained_weights/losses.log

Epoch 00024: val_loss improved from 0.79108 to 0.78966, saving model to ../trained_weights//KERAS_check_best_model.h5

Epoch 00024: val_loss improved from 0.79108 to 0.78966, 

592083/592083 [==============================] - 6s 10us/step - loss: 0.7770 - acc: 0.7419 - val_loss: 0.7751 - val_acc: 0.7424

***callbacks***
saving losses to ../trained_weights/losses.log

Epoch 00036: val_loss improved from 0.77575 to 0.77508, saving model to ../trained_weights//KERAS_check_best_model.h5

Epoch 00036: val_loss improved from 0.77575 to 0.77508, saving model to ../trained_weights//KERAS_check_best_model_weights.h5

Epoch 00036: saving model to ../trained_weights//KERAS_check_model_last.h5

Epoch 00036: saving model to ../trained_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 37/100
592083/592083 [==============================] - 5s 9us/step - loss: 0.7760 - acc: 0.7421 - val_loss: 0.7741 - val_acc: 0.7425

***callbacks***
saving losses to ../trained_weights/losses.log

Epoch 00037: val_loss improved from 0.77508 to 0.77409, saving model to ../trained_weights//KERAS_check_best_model.h5

Epoch 00037: val_loss improved from 0.77508 to 0.77409, 

592083/592083 [==============================] - 5s 9us/step - loss: 0.7678 - acc: 0.7441 - val_loss: 0.7664 - val_acc: 0.7442

***callbacks***
saving losses to ../trained_weights/losses.log

Epoch 00049: val_loss improved from 0.76676 to 0.76642, saving model to ../trained_weights//KERAS_check_best_model.h5

Epoch 00049: val_loss improved from 0.76676 to 0.76642, saving model to ../trained_weights//KERAS_check_best_model_weights.h5

Epoch 00049: saving model to ../trained_weights//KERAS_check_model_last.h5

Epoch 00049: saving model to ../trained_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 50/100
592083/592083 [==============================] - 5s 9us/step - loss: 0.7672 - acc: 0.7443 - val_loss: 0.7660 - val_acc: 0.7442

***callbacks***
saving losses to ../trained_weights/losses.log

Epoch 00050: val_loss improved from 0.76642 to 0.76597, saving model to ../trained_weights//KERAS_check_best_model.h5

Epoch 00050: val_loss improved from 0.76642 to 0.76597, s

592083/592083 [==============================] - 5s 9us/step - loss: 0.7608 - acc: 0.7458 - val_loss: 0.7595 - val_acc: 0.7454

***callbacks***
saving losses to ../trained_weights/losses.log

Epoch 00062: val_loss improved from 0.76016 to 0.75953, saving model to ../trained_weights//KERAS_check_best_model.h5

Epoch 00062: val_loss improved from 0.76016 to 0.75953, saving model to ../trained_weights//KERAS_check_best_model_weights.h5

Epoch 00062: saving model to ../trained_weights//KERAS_check_model_last.h5

Epoch 00062: saving model to ../trained_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 63/100
592083/592083 [==============================] - 5s 9us/step - loss: 0.7603 - acc: 0.7457 - val_loss: 0.7595 - val_acc: 0.7454

***callbacks***
saving losses to ../trained_weights/losses.log

Epoch 00063: val_loss improved from 0.75953 to 0.75952, saving model to ../trained_weights//KERAS_check_best_model.h5

Epoch 00063: val_loss improved from 0.75953 to 0.75952, s

592083/592083 [==============================] - 5s 8us/step - loss: 0.7553 - acc: 0.7467 - val_loss: 0.7544 - val_acc: 0.7465

***callbacks***
saving losses to ../trained_weights/losses.log

Epoch 00075: val_loss improved from 0.75528 to 0.75435, saving model to ../trained_weights//KERAS_check_best_model.h5

Epoch 00075: val_loss improved from 0.75528 to 0.75435, saving model to ../trained_weights//KERAS_check_best_model_weights.h5

Epoch 00075: saving model to ../trained_weights//KERAS_check_model_last.h5

Epoch 00075: saving model to ../trained_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 76/100
592083/592083 [==============================] - 4s 7us/step - loss: 0.7549 - acc: 0.7470 - val_loss: 0.7540 - val_acc: 0.7467

***callbacks***
saving losses to ../trained_weights/losses.log

Epoch 00076: val_loss improved from 0.75435 to 0.75398, saving model to ../trained_weights//KERAS_check_best_model.h5

Epoch 00076: val_loss improved from 0.75435 to 0.75398, s

592083/592083 [==============================] - 6s 9us/step - loss: 0.7506 - acc: 0.7478 - val_loss: 0.7500 - val_acc: 0.7475

***callbacks***
saving losses to ../trained_weights/losses.log

Epoch 00089: val_loss improved from 0.75053 to 0.75001, saving model to ../trained_weights//KERAS_check_best_model.h5

Epoch 00089: val_loss improved from 0.75053 to 0.75001, saving model to ../trained_weights//KERAS_check_best_model_weights.h5

Epoch 00089: saving model to ../trained_weights//KERAS_check_model_last.h5

Epoch 00089: saving model to ../trained_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 90/100
592083/592083 [==============================] - 5s 8us/step - loss: 0.7503 - acc: 0.7478 - val_loss: 0.7502 - val_acc: 0.7471

***callbacks***
saving losses to ../trained_weights/losses.log

Epoch 00090: val_loss did not improve

Epoch 00090: val_loss did not improve

Epoch 00090: saving model to ../trained_weights//KERAS_check_model_last.h5

Epoch 00090: saving mo